In [ ]:
###########################################################
#                                                         #
#               TRAITEMENT SIMUS FORTRAN                  #
#                     STATISTIQUES                        #
#                                                         #
#                         avec                            #
#                   - lecture arborescence                #
#                  analyse                                #
#                   - Delta Fluo                          #
#                   - Delta Energy GMol                   #
#                   - R/L Ca+                             #
#                   - distance inter ionique              #
#                   - trajectoire GMol                    #
#                                                         #
#                          02/2020     Adrien Poindron    #
###########################################################

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
matplotlib.rcParams['figure.figsize'] = [10.5,7]
matplotlib.rcParams.update({'font.size': 25})

# Imports

In [2]:
# %pylab

import numpy
import matplotlib
from matplotlib.patches import Ellipse
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

matplotlib.rcParams.update({'font.size': 21})
# matplotlib.rcParams["figure.dpi"] = 300

from IPython.display import display
from IPython.core.pylabtools import figsize, getfigs

from pylab import *
from numpy import *

from scipy.optimize import curve_fit

In [4]:
import os
from os import listdir
from os.path import isfile, join

import tkinter as tk
from tkinter import filedialog
%matplotlib qt

import matplotlib.patches as patches

from pathlib import Path
from matplotlib import gridspec
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

In [5]:
from natsort import natsorted   # natural sort marche de ouf
import re                       # find '/' in strings
import timeit
cm = pylab.get_cmap('summer')   # colormap par défaut color=cm(k/len(array))

In [157]:
# Fonctions persos de Jofre et Adrien

# %run function_jofre.ipy
%run "../../Functions/1.0/data_fct_Adrien_point_by_point.py"

In [7]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


# Sélection fichier

In [17]:
## GUI for data loading
# Select one data file all the way down to the directories
# Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+00V_D1.0_S1.0RFG.dat

file_cfg, slash_cfg, all_subdir = load_gui(filter_nocomplete=1)

file_path = file_cfg[0]
dir_path = file_cfg[1]
work_rep = file_cfg[2]
filename = file_cfg[3]

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

> Répertoire : /home/adrian/RemoteFS/Hobitton/20200131/DC01_RF08/Try00
> Filename : Temp_SimuType0_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG.dat
> myslashpos | [0, 5, 12, 21, 30, 39, 49]
> slashcond | 5
Points deleted because they were not complete []   0 pt(s)
Total number of data directories 962


In [18]:
## Condition for data
points_and_coord, condition_parameters = simu_conditions(all_subdir, myslashpos, slashcond, filename)

> condition names ['DC', 'RF']
> number of points 74
000 > DC01_RF08
001 > DC01_RF09
002 > DC01_RF10
003 > DC01_RF11
004 > DC01_RF14
005 > DC01_RF15
006 > DC01_RF16
007 > DC01_RF17
008 > DC01_RF18
009 > DC01_RF19
010 > DC01_RF20
011 > DC03_RF08
012 > DC03_RF09
013 > DC03_RF10
014 > DC03_RF11
015 > DC03_RF14
016 > DC03_RF15
017 > DC03_RF16
018 > DC03_RF17
019 > DC03_RF18
020 > DC03_RF19
021 > DC03_RF20
022 > DC05_RF08
023 > DC05_RF09
024 > DC05_RF10
025 > DC05_RF11
026 > DC05_RF14
027 > DC05_RF15
028 > DC05_RF16
029 > DC05_RF17
030 > DC05_RF18
031 > DC07_RF08
032 > DC07_RF09
033 > DC07_RF10
034 > DC07_RF11
035 > DC07_RF14
036 > DC07_RF15
037 > DC07_RF16
038 > DC07_RF17
039 > DC07_RF18
040 > DC09_RF08
041 > DC09_RF09
042 > DC09_RF10
043 > DC09_RF11
044 > DC09_RF14
045 > DC09_RF15
046 > DC09_RF16
047 > DC09_RF17
048 > DC09_RF18
049 > DC10_RF08
050 > DC10_RF09
051 > DC10_RF10
052 > DC10_RF11
053 > DC10_RF14
054 > DC10_RF15
055 > DC10_RF16
056 > DC10_RF17
057 > DC10_RF18
058 > DC11_RF08
059

In [10]:
## Data loading (1D arrays)
data_name, num_runs, PMandT, Gmol_data, r_LC_clip, dim_nu \
= data_retrieve(all_subdir, points_and_coord, condition_parameters, slash_cfg) 

data0 = data_name[0]
data2 = data_name[1]
data4 = data_name[2]

PMvar = PMandT[0]
Tvar = PMandT[1]

deltaEc = Gmol_data[0]
deltaEcRel = Gmol_data[1]
t_c = Gmol_data[2]

NameError: name 'points_and_coord' is not defined

In [21]:
# save_subdir = all_subdir
# all_subdir = all_subdir[0:1];print(all_subdir)

['/home/adrian/RemoteFS/Hobitton/20200131/DC01_RF08/Try00']


In [45]:
all_subdir = save_subdir

In [158]:
def file_name_retrieve(address,simu_type):
        # get only .dat files in each simulation directory
    onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and not "xva" in f and ".dat" in f]
    tmp_file = []
    # build path file
    if simu_type == 'Collision':
        for k in range(3):
            tmp_file.append('{}/{}'.format(address,sort(onlyfiles)[k].strip('.dat')))
    elif simu_type == 'RF_relax':
        for k in range(4):
            try:
                tmp_file.append('{}/{}'.format(address,sort(onlyfiles)[k].strip('.dat')))
            except(IndexError):
                tmp_file.append('No file for this SimuType')
                print('A file is missing, skiping the file number',k)
            
    return onlyfiles, tmp_file

In [159]:
def cloud_size(address,onlyfiles, **kwargs):
    radial_thresh = kwargs.get('radial_thresh', 10e-2)
    axial_thresh  = kwargs.get('axial_thresh', 1e-0)
    
    try:
        my_file = '{}/xva{}'.format(address,sort(onlyfiles)[0].strip('.dat')[4:])
        r_LC,v_LC,a_LC = load_xyz_init_bin_DP(my_file)

        # filter lost ions
        x_LC_clip = [r_LC[0,x] for x in range(len(r_LC[0,:])) if abs(r_LC[0,x]) < radial_thresh] #6e-2
        y_LC_clip = [r_LC[1,x] for x in range(len(r_LC[1,:])) if abs(r_LC[1,x]) < radial_thresh]
        z_LC_clip = [r_LC[2,x] for x in range(len(r_LC[2,:])) if abs(r_LC[2,x]) < axial_thresh]
#         r_LC_clip[k][j][:] = [x_LC_clip,y_LC_clip,z_LC_clip]
        r_LC_clip = [x_LC_clip,y_LC_clip,z_LC_clip]
        dim_nu = [max(r_LC_clip[l][:]) for l in range(3)]
    except:
        dim_nu = None
            
    return r_LC_clip, dim_nu

In [165]:
def load_RF_relax(address,flag_plot):
    # retrieve files adresses
    onlyfiles, data_files = file_name_retrieve(address,simu_type='RF_relax')
    file0 = data_files[0];file2 = data_files[1];file4 = data_files[2];file6 = data_files[3]
    # load Ec variation for GMol
    _,_,deltaEc,deltaEcRel,t_c = energy_lost(address,'xva'+sort(onlyfiles)[2][4:].strip('.dat'))
    # load cloud size before injection
    r_LC_clip, dim_nu = cloud_size(address,onlyfiles)
    # load fluorescence and T var
    # Temperature during time
#     tt, T_aux, PM, PMvar, Tvar, SNR = plot_T_and_PM_Init_RFrelax_AfterInj_Evol(address+'/',sort(onlyfiles)[0].strip('.dat')[4:],1,'RFrelax_AfterInj',xlim1=(3.42,4.45),ylim1=(5e-5,3e3))
    temps, temperature, fluo = plot_T_and_PM_Init_RFrelax_AfterCooling_Evol(address+'/',sort(onlyfiles)[0].strip('.dat')[4:],flag_plot,'RFrelax_AfterCool',xlim1=(-1.5,55),ylim1=(5e-5,12e3))
    
    fluo_var = [fluo[1], temperature[1], fluo[2]]        # temps = [tt,t_aux1, t_aux2]
    GMol_var = [deltaEc,deltaEcRel,t_c]                  # fluo = [PM, PM_variation, SNR]
    cloud_atlas = [temps[0], temperature[0], fluo[0]]    # temperature = [T_aux, T_variation]
    t_aux = [temps[1],temps[2]]
    
    return fluo_var, GMol_var, cloud_atlas, t_aux, r_LC_clip

In [161]:
all_subdir[0][myslashpos[slashcond]:myslashpos[slashcond+1]]

'/DC01_RF08'

In [167]:
# def data_retrieve(all_subdir[0:2],points_and_coord, condition_parameters, slash_cfg):

# Récupérer les données pour chaque simu
# Exécution en quelques minutes

# data0, data2, data4 : nom des fichiers de fluo et température
# PMvar, Tvar         : variation de fluo et température entre passage Gmol et fin de simu
# deltaEc, deltaEcRel : variation énergie GMol
# t_c                 : temps de vol de la GMol entre son apparition (z=-1.5mm) et sa disparition (z=1.5mm)
# r_LC, v_LC, a_LC    : position, vitesse, accélération des ions Ca+
# r_LC_clip           : même chose mais en dégageant les ions perdus et beaucoup trop loin (hors pièges)
# dim_nu              : dimensions x, y et z, nuage Ca+ basé sur r_LC_clip

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

# determining number of elements on each repetition
num_runs = [runs[myslashpos[slashcond+1]+1:] for runs in all_subdir if list(points_and_coord.keys())[0] in runs]
num_runs = list(dict.fromkeys(num_runs))

# number of repetitions
# print('> Points |',len(points_and_coord))
# print('> Simulations pour chaque point |', num_runs)

# file0 = [[] for i in range(len(num_runs))] # file path to SimuType0
# file2 = [[] for i in range(len(num_runs))] # file path to SimuType2
# file4 = [[] for i in range(len(num_runs))] # file path to SimuType4
# file6 = [[] for i in range(len(num_runs))] # file path to SimuType6
# data_address = [[] for i in range(len(num_runs))]

# Variables à deux coordonnées : [point, try]

simutype6address = []
pts_concerned = []
for k, address in enumerate(all_subdir):
    onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and not "xva" in f and ".dat" in f]
    onlySimuType6 = [i for i in onlyfiles if 'SimuType6' in i]
    print(k,'- ', onlySimuType6)
    if onlySimuType6 != []:
        simutype6address.append(address)
        pts_concerned.append( re.sub('/','',all_subdir[k][myslashpos[slashcond]:myslashpos[slashcond+1]]) )
fluo_var, GMol_var, cloud_atlas, t_aux, r_LC_clip = load_RF_relax(all_subdir[0],flag_plot=1)
        
shapevar = (len(simutype6address),len(cloud_atlas[2]))
PMvar = np.zeros(shapevar)
Tvar = np.zeros(shapevar)
SNR = np.zeros(shapevar)
tt = np.zeros(shapevar)
T_aux = np.zeros((len(simutype6address),len(cloud_atlas[2]),3))
PM = np.zeros(shapevar)
        
t0 = time.clock()
print("Hello")
    
for k, address in enumerate(simutype6address):
    fluo_var, GMol_var, cloud_atlas, t_aux, r_LC_clip = load_RF_relax(address,flag_plot=1)
    PMvar[k] = fluo_var[0] ; Tvar[k] = fluo_var[1] ; SNR[k] = fluo_var[2]
    tt[k] = cloud_atlas[0] ; T_aux[k] = cloud_atlas[1] ; PM[k] = cloud_atlas[2]

t1 = time.clock() - t0
print("Time elapsed: ", t1, 's') # CPU seconds elapsed (floating point)
print("Time elapsed: ", t1/60, 'm') # CPU seconds elapsed (floating point)

0 -  ['Temp_SimuType6_01_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG.dat']
1 -  []
2 -  []
3 -  []
4 -  []
5 -  []
6 -  []
7 -  []
8 -  []
9 -  []
10 -  []
11 -  []
12 -  []
13 -  ['Temp_SimuType6_01_N01024_Vrf0059_Udc0.1000D+01V_D1.0_S1.0RFG.dat']
14 -  []
15 -  []
16 -  []
17 -  []
18 -  []
19 -  []
20 -  []
21 -  []
22 -  []
23 -  []
24 -  []
25 -  []
26 -  ['Temp_SimuType6_01_N01024_Vrf0064_Udc0.1000D+01V_D1.0_S1.0RFG.dat']
27 -  []
28 -  []
29 -  []
30 -  []
31 -  []
32 -  []
33 -  []
34 -  []
35 -  []
36 -  []
37 -  []
38 -  []
39 -  ['Temp_SimuType6_01_N01024_Vrf0070_Udc0.1000D+01V_D1.0_S1.0RFG.dat']
40 -  []
41 -  []
42 -  []
43 -  []
44 -  []
45 -  []
46 -  []
47 -  []
48 -  []
49 -  []
50 -  []
51 -  []
52 -  []
53 -  []
54 -  []
55 -  []
56 -  []
57 -  []
58 -  []
59 -  []
60 -  []
61 -  []
62 -  []
63 -  []
64 -  []
65 -  []
66 -  []
67 -  []
68 -  []
69 -  []
70 -  []
71 -  []
72 -  []
73 -  []
74 -  []
75 -  []
76 -  []
77 -  []
78 -  []
79 -  []
80 -  []
81 -  []
82 -  []

709 -  []
710 -  []
711 -  []
712 -  []
713 -  []
714 -  []
715 -  []
716 -  []
717 -  []
718 -  []
719 -  []
720 -  []
721 -  []
722 -  []
723 -  []
724 -  []
725 -  []
726 -  []
727 -  []
728 -  []
729 -  []
730 -  []
731 -  []
732 -  []
733 -  []
734 -  []
735 -  []
736 -  []
737 -  []
738 -  []
739 -  []
740 -  []
741 -  []
742 -  []
743 -  []
744 -  []
745 -  []
746 -  []
747 -  []
748 -  []
749 -  []
750 -  []
751 -  []
752 -  []
753 -  []
754 -  ['Temp_SimuType6_01_N01024_Vrf0053_Udc0.7000D+01V_D1.0_S1.0RFG.dat']
755 -  []
756 -  []
757 -  []
758 -  []
759 -  []
760 -  []
761 -  []
762 -  []
763 -  []
764 -  []
765 -  []
766 -  []
767 -  ['Temp_SimuType6_01_N01024_Vrf0059_Udc0.7000D+01V_D1.0_S1.0RFG.dat']
768 -  []
769 -  []
770 -  []
771 -  []
772 -  []
773 -  []
774 -  []
775 -  []
776 -  []
777 -  []
778 -  []
779 -  []
780 -  ['Temp_SimuType6_01_N01024_Vrf0064_Udc0.7000D+01V_D1.0_S1.0RFG.dat']
781 -  []
782 -  []
783 -  []
784 -  []
785 -  []
786 -  []
787 -  []
788 -  []
78

In [174]:
# Plot all temperatures rf relax après refroidissement

fig, ax = plt.subplots(4, 7,num='Temperature RF relax sans injection',clear=True,sharex=True,sharey=True)
fig.set_size_inches(11.69,8.27)
# suptitle('Saturation 866')
to_plot = 0
for l in range(0,7):
    for k in range(0,4):
        ax[3-k,l].semilogy(tt[to_plot,:]*1e3,T_aux[to_plot,:,0],'+',mew=1,ms=5)
        ax[3-k,l].semilogy(tt[to_plot,:]*1e3,T_aux[to_plot,:,1],'+',mew=1,ms=5)
        ax[3-k,l].semilogy(tt[to_plot,:]*1e3,T_aux[to_plot,:,2],'+',mew=1,ms=5)
        ax[3-k,l].vlines(t_aux[0]*1e3,1e-4,1e-1,color='red')
        ax[3-k,l].vlines(t_aux[1]*1e3,1e-4,1e-1,color='red')
        ax[3-k,l].set_yticks([1e-4,1e-3,1e-2,1e-1,1,1e2,1e4])
        ax[3-k,l].grid()
    #     ax[k].set_ylim(-0.1,0.55)
        ax[3-k,l].xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=2))
        ax[3-k,l].annotate(pts_concerned[to_plot], xy=(0.29, 0.99), xycoords='axes fraction',
                size=12, ha='left', va='top',
                bbox=dict(boxstyle='round', fc='w'))
        to_plot += 1
# ax[3].set_xlim(-10,4000)
ax[3-k,l].set_ylim(2e-4,1e-1)
fig.text(0.56, 0.04, 'time [ms]', ha='center')
fig.text(0.04, 0.54, 'T [K]', va='center', rotation='vertical')

fig.suptitle(all_subdir[0],fontsize=10)

# fig.text(0.5, 0.01, 'f', ha='center')
# fig.text(0.01, 0.5, r'$| FFT(HC(t)) | / N_{sample}$', va='center', rotation='vertical')

plt.tight_layout()
subplots_adjust(hspace=0.05, wspace = 0.05, bottom = 0.120, left = 0.140) # hspace=0, top=0.95, left = 0.065, bottom = 0.07
# savefig('FFT_lock_oscillo_0_11500.png')
# fig.set_size_inches(11.69,8.27)

/home/adrian/.local/lib/python3.6/site-packages/matplotlib/axes/_base.py:3449: UserWarning: Attempted to set non-positive ylimits for log-scale axis; invalid limits will be ignored.
  'Attempted to set non-positive ylimits for log-scale axis; '


In [286]:
fluo_var_one, GMol_var_one, cloud_atlas_one, t_aux_one, r_LC_clip_one = load_RF_relax(simutype6address[25],flag_plot=1)
T_aux = cloud_atlas_one[1]
T_aux = T_aux[len(freq_one)//3:,1]
time_one = cloud_atlas_one[0]
dtime_one = time_one[50]-time_one[49]
freq_one = np.linspace(0,1/2/dtime_one,len(T_aux))
print(dtime_one,freq_one[51]-freq_one[50])
fig = figure('FFT'); clf()
ax1 = subplot(111)
ax1.scatter(freq_one*1e-3,abs(real(fft(T_aux*hanning(len(T_aux))))),marker='+')
ax1.grid()

legend()
xlabel('f [kHz]')

fig.set_size_inches(11.69, 8.27)
plt.tight_layout()

No handles with labels found to put in legend.


4.926829268293804e-07 79.92844649510016


In [264]:
len(freq_one)//3

4232

In [183]:
for k, l in enumerate(simutype6address):
    print('>',k,'-',l)

> 0 - /home/adrian/RemoteFS/Hobitton/20200131/DC01_RF08/Try00
> 1 - /home/adrian/RemoteFS/Hobitton/20200131/DC01_RF09/Try00
> 2 - /home/adrian/RemoteFS/Hobitton/20200131/DC01_RF10/Try00
> 3 - /home/adrian/RemoteFS/Hobitton/20200131/DC01_RF11/Try00
> 4 - /home/adrian/RemoteFS/Hobitton/20200131/DC03_RF08/Try00
> 5 - /home/adrian/RemoteFS/Hobitton/20200131/DC03_RF09/Try00
> 6 - /home/adrian/RemoteFS/Hobitton/20200131/DC03_RF10/Try00
> 7 - /home/adrian/RemoteFS/Hobitton/20200131/DC03_RF11/Try00
> 8 - /home/adrian/RemoteFS/Hobitton/20200131/DC05_RF08/Try00
> 9 - /home/adrian/RemoteFS/Hobitton/20200131/DC05_RF09/Try00
> 10 - /home/adrian/RemoteFS/Hobitton/20200131/DC05_RF10/Try00
> 11 - /home/adrian/RemoteFS/Hobitton/20200131/DC05_RF11/Try00
> 12 - /home/adrian/RemoteFS/Hobitton/20200131/DC07_RF08/Try00
> 13 - /home/adrian/RemoteFS/Hobitton/20200131/DC07_RF09/Try00
> 14 - /home/adrian/RemoteFS/Hobitton/20200131/DC07_RF10/Try00
> 15 - /home/adrian/RemoteFS/Hobitton/20200131/DC07_RF11/Try00
> 